In [77]:
import os
import openai
from dotenv import load_dotenv
# import pandas as pd
# pd.DataFrame(openai.Model.list()['data'])

In [78]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [79]:
response=openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo',
    messages = [{'role':'system','content':'you are assistant which give temperature information'},
                    {'role':'user','content':'hey there'},
                ]
)


In [80]:
print(completion.choices[0].message)

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"Bangalore\"\n}"
  }
}


In [2]:
##copy code from rapidAPI
import requests
import json
def get_current_weather(location):
    """get the current weather location"""
    url = "https://ai-weather-by-meteosource.p.rapidapi.com/find_places"
    querystring = {"text":location}
    headers = {
	"X-RapidAPI-Key": "8f423c8942mshea41da156cb8838p1f3736jsn48a371fd0a23",
	"X-RapidAPI-Host": "ai-weather-by-meteosource.p.rapidapi.com"
 }

    response = requests.get(url, headers=headers, params=querystring)
    print(type(response))
    return response.json()


In [4]:
response=get_current_weather('banglore')

<class 'requests.models.Response'>


[]

In [83]:
functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                },
                "required": ["location"],
            },
        }
    ]

In [84]:
functions

[{'name': 'get_current_weather',
  'description': 'Get the current weather in a given location',
  'parameters': {'type': 'object',
   'properties': {'location': {'type': 'string',
     'description': 'The city and state, e.g. San Francisco, CA'}},
   'required': ['location']}}]

In [85]:
user_message="hi there.."
messages =[]
messages.append({'role':'user','content':user_message})
completion=openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=messages
)
print(completion.choices[0].message)
messages

{
  "role": "assistant",
  "content": "Hello! How can I assist you today?"
}


[{'role': 'user', 'content': 'hi there..'}]

In [86]:
city=input('enter city:')
user_message=f"what is the temperature of {city}"
messages.append({'role':'user','content':user_message})
completion=openai.ChatCompletion.create(
    model='gpt-3.5-turbo-0613',
    messages=messages,
    functions=functions
)
print(completion.choices[0].message)


{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"Bangalore\"\n}"
  }
}


In [87]:
response=completion.choices[0].message
function_name=response['function_call']['name']
print(function_name)

get_current_weather


In [88]:
import json
location = eval(response['function_call']['arguments'])['location']
print(location)

Bangalore


In [89]:
# send the info on the function call and function response to GPT
messages.append(response)
messages.append(
    {
        'role':'function',
        'name':function_name,
        'content':location
    }
)
messages

[{'role': 'user', 'content': 'hi there..'},
 {'role': 'user', 'content': 'what is the temperature of banglore'},
 <OpenAIObject at 0x168923afdd0> JSON: {
   "role": "assistant",
   "content": null,
   "function_call": {
     "name": "get_current_weather",
     "arguments": "{\n  \"location\": \"Bangalore\"\n}"
   }
 },
 {'role': 'function', 'name': 'get_current_weather', 'content': 'Bangalore'}]

In [90]:
second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
            functions=functions
)

In [91]:
print(second_response.choices[0].message)

{
  "role": "assistant",
  "content": "The current temperature in Bangalore is 25 degrees Celsius."
}
